In [20]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [21]:
# Hyperparameters

input_size = 784 # image size = (28,28) ie. 28*28=784 in total
hidden_size = 500 # number of nodes in the hidden layer
num_classes = 10 # number of output classes (10 digits)
num_epochs = 10 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 0.001 # learning rate

In [22]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [23]:
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [24]:
class model(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(model,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [25]:
net = model(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
    net.cuda()

In [26]:
#Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [27]:
#Training the model
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_loader):
    batch_loss = 0 # loss at each batch
    
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    
    if (i+1) % 100 == 0:
      print('Epoch %d | Step %d | Loss: %.4f' %(epoch+1, i+1, batch_loss))

Epoch 1 | Step 100 | Loss: 0.3964
Epoch 1 | Step 200 | Loss: 0.1945
Epoch 1 | Step 300 | Loss: 0.2658
Epoch 1 | Step 400 | Loss: 0.1340
Epoch 1 | Step 500 | Loss: 0.2339
Epoch 1 | Step 600 | Loss: 0.1112
Epoch 2 | Step 100 | Loss: 0.0791
Epoch 2 | Step 200 | Loss: 0.0885
Epoch 2 | Step 300 | Loss: 0.0477
Epoch 2 | Step 400 | Loss: 0.0947
Epoch 2 | Step 500 | Loss: 0.1204
Epoch 2 | Step 600 | Loss: 0.1128
Epoch 3 | Step 100 | Loss: 0.1780
Epoch 3 | Step 200 | Loss: 0.0529
Epoch 3 | Step 300 | Loss: 0.0477
Epoch 3 | Step 400 | Loss: 0.0477
Epoch 3 | Step 500 | Loss: 0.0897
Epoch 3 | Step 600 | Loss: 0.1458
Epoch 4 | Step 100 | Loss: 0.0669
Epoch 4 | Step 200 | Loss: 0.0630
Epoch 4 | Step 300 | Loss: 0.0417
Epoch 4 | Step 400 | Loss: 0.0204
Epoch 4 | Step 500 | Loss: 0.0417
Epoch 4 | Step 600 | Loss: 0.0940
Epoch 5 | Step 100 | Loss: 0.0376
Epoch 5 | Step 200 | Loss: 0.0134
Epoch 5 | Step 300 | Loss: 0.0149
Epoch 5 | Step 400 | Loss: 0.0627
Epoch 5 | Step 500 | Loss: 0.0164
Epoch 5 | Step

In [28]:
#Accuracy of the model
 
correct = 0
total = 0
for images,labels in test_loader:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)
 
print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 98.020 %
